# Beispiel 1.5
Im Folgenden lösen wir das lineare Optimierungsproblem aus Beispiel 1.5 aus der Vorlesung.
Dieses lautet allgemein

$
\begin{align*}
\min \; x_{n+1} \quad \text{ u.d.N. } \quad -\left(f_j - \sum \limits_{i = 1}^n x_i v_i(t_j) \right) &\leq x_{n+1} \;\forall j \in [m]\\
f_j - \sum \limits_{i = 1}^n x_i v_i(t_j) &\leq x_{n+1} \;\forall j \in [m]\\
x &\in \mathbb{R}^{n+1}.
\end{align*}
$

Wir definieren nun die Hilfsvariablen

$
B = \begin{pmatrix}v_1(t_1) & \ldots & v_n(t_1) \\ \vdots & & \vdots \\ v_1(t_m) & \ldots & v_n(t_m) \end{pmatrix} \in \mathbb{R}^{m \times n}, \quad e = \begin{pmatrix}1 \\ \ldots \\ 1 \end{pmatrix} \in \mathbb{R}^m, \quad d = \begin{pmatrix}f_1 \\ \ldots \\ f_m \end{pmatrix} \in \mathbb{R}^m.
$

Damit ergibt sich als LP zu obigem Problem

$
\begin{align*}
\min \; c^\top x \quad \text{ u.d.N. } Ax &\leq b\\
x &\in \mathbb{R}^{n+1}.
\end{align*}
$

mit $c = \begin{pmatrix}0\\ \vdots \\ 0 \\ 1 \end{pmatrix} \in \mathbb{R}^{n+1}, \quad A = \begin{pmatrix}B & -e \\ -B & -e\end{pmatrix} \in \mathbb{R}^{(2m) \times (n+1)}, \quad b = \begin{pmatrix} d \\ -d \end{pmatrix} \in \mathbb{R}^{2m}$.

Im Folgenden betrachten wir konkret $n = 4,\; m = 5,\; t = (1,2,3,4,5)^\top,\; f = (3,1,2,1,4)^\top$.

Zunächst werden benötigte Pakete und Funktionen geladen:

In [ ]:
from scipy.optimize import linprog
import numpy as np
import matplotlib.pyplot as plot

Anschließend legen wir die Parameter fest.
Hierbei gibt es nun jedoch einiges zu beachten.
Wir "bauen" in diesem Beispiel zum Teil schon etwas komplexere Gebilde.
So wird $A$ aus $B$ und $e$ konstruiert und auch für die Monome $v_i(t_j)$ werden wir uns eine clevere Struktur überlegen müssen.
Die gewöhnlichen Python Arrays stoßen hier an ihre Grenzen.
Deshalb gibt es eigene Arrays für die Verwendung in numpy, die im folgenden Code mit `np.array` erzeugt werden.
Diese erleichtern uns die Arbeit.

Nun stellt sich noch die Frage, wie wir die Monome verwalten.
Funktionen wären an dieser Stelle etwas überdimensioniert, denn für jedes der 4 Monome benötigen wir nur 5 Werte.
Deshalb haben wir uns hier dafür entschieden, die entsprechenden Werte in eine Matrix $V \in \mathbb{R}^{n \times m}$ zu schreiben.
Diese enthält in der $i$-ten Zeile gerade $(t_1^i, \ldots t_m^i)$.
In numpy müssen wir glücklicherweise nicht jeden Wert $t_j$ einzeln in eine bestimmte Potenz erheben, sondern können mit `np.power` den ganzen Vektor $t$ verarbeiten.

An dieser Stelle sei wieder angemerkt, dass wir die Vektoren als Zeilenvektoren angelegt haben.
Das müssen wir natürlich immer berücksichtigen und gegebenenfalls mit `np.transpose` Vektoren und Matrizen transponieren.
Wer möchte, kann den Code gerne auch so anpassen, dass die Variablen `t,f` von Anfang an Spaltenvektoren sind.

Nun aber zum entsprechenden Code:

In [ ]:
m = 5
n = 4
t = np.array([1, 2, 3, 4, 5])
f = np.array([3, 1, 2, 1, 4])
V = np.array([np.power(t,0),np.power(t,1),np.power(t,2),np.power(t,3)])
B = np.transpose(V)
d = f
e = np.ones((m,1))

Jetzt müssen wir diese Parameter noch so miteinander verknüpfen, dass wir $A, b$ und $c$ für unser Optimierungsproblem erhalten.
Im Englischen lässt sich verknüpfen mit *concatenate* übersetzen und entsprechend können wir mit dem Befehl `np.concatenate` auch genau das tun.
Wie in der [Dokumentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html) nachgelesen werden kann, sollte dazu auch die Richtung (Axe) angegeben werden, entlang welcher diese Verknüpfung stattfinden soll.
Dabei gilt es zwei Dinge zu beachten: Zum einen wird in Python immer bei 0 angefangen zu zählen (in MATLAB bei 1) und deshalb hat eine zweidimensionale Matrix die Axenrichtungen 0 und 1.
Zum anderen kann für `axis=0` dieser zweite Parameter auch ausgelassen werden.
Im nachfolgenden Code ist das für $b$ gemacht worden.

Nun ist der `np.concatenate` Befehl relativ weit entfernt von der gewohnten MATLAB Syntax zur Verknüpfung von Arrays.
Daher gibt es auch den Befehl `np.block`, der sehr viel näher an dieser Syntax liegt.
Es sei an dieser Stelle angemerkt, dass `np.concatenate` sich in der Praxis als der schnellste Weg herausgestellt hat, um Arrays zu verknüpfen.
Für unsere Zwecke macht sich dieser Geschwindigkeitsunterschied allerdings kaum bemerkbar und wir haben uns für die besser lesbare Variante mit `np.block` entschieden, um die Matrix $A$ zu generieren.

Für das LP erhalten wir nun also folgende Parameter:

In [ ]:
c = np.zeros((1,n))
c = np.append(c,1)
#A = np.concatenate((np.concatenate((B,-e),axis=1),np.concatenate((-B,-e),axis=1)),axis=0)
A = np.block([[B,-e],[-B,-e]])
b = np.concatenate((d,-d))

Nun folgt wie gewohnt der Aufruf von `linprog` und anschließend eine Ausgabe der Ergebnisse:

In [ ]:
res = linprog(c, A_ub=A, b_ub=b, bounds=(None,None))
print("Optimallösung: " + str(res.x))
print("Optimalwert: " + str(res.fun))

Zum Abschluss noch eine grafische Darstellung der gegebenen Werte und der von uns bestimmten Approximation.
Man beachte dabei die geringfügigen Unterschiede bei der Übergabe der Parameter an den `plot` Befehl im Vergleich zu MATLAB:

In [ ]:
plot.plot(t,f,'o',ls='')
s=np.linspace(0,6,50);
y=res.x[0]*1+res.x[1]*s+res.x[2]*np.power(s,2)+res.x[3]*np.power(s,3);
plot.plot(s,y,linewidth=2,color='orange')

plot.title('Approximation für Beispiel 1.5')
plot.xlabel('t');
plot.ylabel('f');
plot.legend(['Vorgegebene Werte (Messwerte)','Approximation (über LP)']);